In [1]:
import pickle
from pprint import pprint
import psycopg2

In [4]:
dag_ids = ['telephone_game_1']
READ_COST_TEMPLATE = "./include/dag_timings/{}_timing_logs.pkl"
TOTAL_COST_TEMPLATE = "./include/dag_timings/{}_task_durations.pkl"

read_costs = {}
total_costs = {}
for dag_id in dag_ids:
    read_cost_file = READ_COST_TEMPLATE.format(dag_id)
    total_cost_file = TOTAL_COST_TEMPLATE.format(dag_id)
    with open(read_cost_file, "rb") as f:
        loaded_data = pickle.load(f)
        read_costs[dag_id] = loaded_data
    with open(total_cost_file, "rb") as f:
        loaded_data = pickle.load(f)
        total_costs[dag_id] = loaded_data

print(f"Files: {read_cost_file}, {total_cost_file}")
print('READ COSTS:')
pprint(read_costs)
print('TOTAL COSTS:')
pprint(total_costs)


Files: ./include/dag_timings/FinSum_OpenAI_timing_logs.pkl, ./include/dag_timings/FinSum_OpenAI_task_durations.pkl
READ COSTS:
{'FinSum_OpenAI': {'extract': {('xcom', 'df', 'write'): 0.2053971290588379},
                   'split': {('xcom', 'df', 'read'): 0.05860781669616699,
                             ('xcom', 'df', 'write'): 0.13010454177856445},
                   'summarize_openai': {('xcom', 'df', 'read'): 0.10287117958068848,
                                        ('xcom', 'summary', 'write'): 0.02402186393737793},
                   'vectorize_chunks': {('xcom', 'df', 'read'): 0.09333229064941406},
                   'vectorize_summaries': {('xcom', 'summary', 'read'): 0.018563508987426758}}}
TOTAL COSTS:
{'FinSum_OpenAI': {'extract': 132.152426,
                   'split': 13.987110000000001,
                   'summarize_openai': 2712.72439,
                   'vectorize_chunks': 1009.146254,
                   'vectorize_summaries': 0.089884}}


In [2]:
dag_id = 'example'
POSTGRES_CONFIG = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',  # will be updated dynamically to point to container IP
    'port': 5432,
}

In [14]:
conn = psycopg2.connect(**POSTGRES_CONFIG)

dag_id = 'FinSum_OpenAI'
query = """
        SELECT task_id, AVG(duration) as total_duration
        FROM task_instance
        WHERE dag_id = %s AND state = 'success'
        GROUP BY task_id;
    """
with conn.cursor() as cur:
    cur.execute(query, (dag_id,))
    results = cur.fetchall()
    column_names = [desc[0] for desc in cur.description]

conn.close()
results

[('split', 2.331185),
 ('vectorize_summaries', 0.06823270000000001),
 ('summarize_openai', 452.12073166666664),
 ('extract', 22.02540433333333),
 ('vectorize_chunks', 168.19104233333334)]

In [11]:
conn = psycopg2.connect(**POSTGRES_CONFIG)
with conn.cursor() as cur:
    cur.execute("SELECT state FROM dag_run WHERE dag_id = %s ORDER BY execution_date DESC LIMIT 1", ('blah',))
    state = cur.fetchone()
conn.close()
print(state[0])

TypeError: 'NoneType' object is not subscriptable

In [9]:
#Print contenets from pkl file
import pickle
with open('./include/telephone_game_1.pkl', 'rb') as f:
    loaded_data = pickle.load(f)
    print(loaded_data)

([], [('upstream', 'misunderstand_task1'), ('misunderstand_task1', 'misunderstand_task2'), ('misunderstand_task2', 'misunderstand_task3'), ('misunderstand_task3', 'misunderstand_task4'), ('misunderstand_task4', 'downstream')])
